In [7]:
import boto3
import pandas as pd
import os
import json


In [9]:
import requests
import json
from datetime import datetime
import os

# --- CONFIGURATION ---
CITY = "Karachi"
API_KEY = "274f03e61fad9fd40610f133518aea20"  # OpenWeatherMap API Key
OUTPUT_FILE = "./historical_aqi_weather_data.json"

# --- COORDINATES FETCH ---
def fetch_coordinates():
    GEO_URL = f"http://api.openweathermap.org/geo/1.0/direct?q={CITY}&limit=1&appid={API_KEY}"
    response = requests.get(GEO_URL)
    response.raise_for_status()
    data = response.json()
    if not data:
        raise Exception("City not found.")
    return data[0]['lat'], data[0]['lon']

# --- CURRENT AQI FETCH ---
def fetch_current_aqi(lat, lon):
    AQI_URL = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}"
    response = requests.get(AQI_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print("❌ Failed to fetch AQI data")
        return None

# --- CURRENT WEATHER FETCH (matching exact UTC hour) ---
def fetch_current_weather(lat, lon):
    OWM_URL = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,"
        f"wind_direction_10m,surface_pressure,cloud_cover,visibility,dew_point_2m,"
        f"apparent_temperature,shortwave_radiation,et0_fao_evapotranspiration"
        f"&forecast_days=1"
    )
    
    response = requests.get(OWM_URL)
    if response.status_code == 200:
        data = response.json()
        hourly = data.get("hourly", {})
        times = hourly.get("time", [])

        # Find the forecast matching current UTC hour
        current_hour_utc = datetime.utcnow().replace(minute=0, second=0, microsecond=0)
        index = None
        for i, t in enumerate(times):
            forecast_time = datetime.strptime(t, "%Y-%m-%dT%H:%M")
            if forecast_time == current_hour_utc:
                index = i
                break

        if index is None:
            print("⚠️ Current hour weather forecast not available yet.")
            return None
        
        return {
            "temperature_2m": hourly.get("temperature_2m", [None])[index],
            "relative_humidity_2m": hourly.get("relative_humidity_2m", [None])[index],
            "precipitation": hourly.get("precipitation", [None])[index],
            "wind_speed_10m": hourly.get("wind_speed_10m", [None])[index],
            "wind_direction_10m": hourly.get("wind_direction_10m", [None])[index],
            "surface_pressure": hourly.get("surface_pressure", [None])[index],
            "cloudcover": hourly.get("cloud_cover", [None])[index],
            "visibility": hourly.get("visibility", [None])[index],
            "dew_point_2m": hourly.get("dew_point_2m", [None])[index],
            "apparent_temperature": hourly.get("apparent_temperature", [None])[index],
            "shortwave_radiation": hourly.get("shortwave_radiation", [None])[index],
            "et0_fao_evapotranspiration": hourly.get("et0_fao_evapotranspiration", [None])[index],
            "timestamp": current_hour_utc.timestamp()
        }
    else:
        print(f"❌ Failed to fetch weather data: {response.text}")
        return None

# --- APPEND DATA TO S3 ---
def append_to_json_array(bucket_name, data_key, new_entry):
    """
    Appends a new entry to JSON array stored in S3.
    
    Args:
        bucket_name: S3 bucket name
        data_key: S3 object key for the JSON file
        new_entry: New data entry to append
    """
    # Initialize S3 client
    s3 = boto3.client(
        's3',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
    )
    
    # Try to get existing data from S3
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=data_key)
        data = json.loads(obj['Body'].read().decode('utf-8'))
        if not isinstance(data, list):
            data = []
    except (s3.exceptions.NoSuchKey, json.JSONDecodeError):
        # File doesn't exist or is not valid JSON
        data = []
    
    # Append new entry
    data.append(new_entry)
    
    # Upload updated data back to S3
    s3.put_object(
        Bucket=bucket_name,
        Key=data_key,
        Body=json.dumps(data, indent=4),
        ContentType='application/json'
    )
    
    print(f"✅ Successfully appended data to S3 bucket: {bucket_name}/{data_key}")

# --- MAIN ---
if __name__ == "__main__":
    try:
        # S3 configuration
        bucket_name = 'my-feature-store-data'
        data_key = 'raw-data/historical_aqi_weather_data.json'
        lat, lon = fetch_coordinates()
        print(f"📍 Coordinates: {lat}, {lon}")
        
        aqi_data = fetch_current_aqi(lat, lon)
        weather_data = fetch_current_weather(lat, lon)
        
        if aqi_data and weather_data:
            entry = {
                "aqi": aqi_data,
                "weather": weather_data
            }
            append_to_json_array(bucket_name,data_key, entry)
            print(f"✅ Successfully saved current data with agricultural parameters")
        else:
            print("⚠️ Failed to collect complete dataset")
    except Exception as e:
        print(f"🚨 Critical error: {e}")


📍 Coordinates: 24.8546842, 67.0207055
✅ Successfully appended data to S3 bucket: my-feature-store-data/raw-data/historical_aqi_weather_data.json
✅ Successfully saved current data with agricultural parameters
